In [66]:
import numpy as np
import torch
import torch.nn as nn


class BatchNorm1d(nn.Module):

    def __init__(self, num_features, eps=1e-05, momentum=0.1):
        super().__init__()
        shape = (1, num_features)

        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))

        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

        self.eps = eps
        self.momentum = momentum

    def forward(self, x):
        # torch.is_grad_enabled() возвращает True, если расчёт градиентов включен, 
        # то есть модель находится в состоянии обучения (train)
        if not torch.is_grad_enabled():
            return self.gamma*(x - self.moving_mean)/np.sqrt(self.moving_var + self.eps) + self.beta
        else:
            neuron_mean = x.mean(dim=0)
            self.moving_mean = (1 - self.momentum)*self.moving_mean + self.momentum*neuron_mean
            neuron_var_bias = x.var(dim=0, unbiased=False)
            neuron_var_unbias = x.var(dim=0, unbiased=True)
            self.moving_var = (1 - self.momentum)*self.moving_var + self.momentum*neuron_var_unbias
            z = (x - neuron_mean)/np.sqrt(neuron_var_bias + self.eps)
            return self.gamma*z + self.beta

def read_matrix(n_rows, dtype=float):
    return np.array([list(map(dtype, input().split())) for _ in range(n_rows)]).astype(float)

def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))

def solution():
    batch_size, num_features = 2, 4
    eps, momentum = 0.01, 0.1
    train_vectors = np.array([[0.5, 1.0, 0.3, 1.3], [0.9, 0.2, 0.4, 1.0]])
    test_vectors = np.array([[0.2, 1.1, 0.5, 1.2], [0.4, 0.7, 0.6, 1.1]])

    train_vectors = torch.from_numpy(train_vectors).float()
    test_vectors = torch.from_numpy(test_vectors).float()

    batch_norm_1d = BatchNorm1d(num_features, eps, momentum)
    output_train = batch_norm_1d.forward(train_vectors).detach().numpy().round(3)
    with torch.no_grad():
        output_eval = batch_norm_1d.forward(test_vectors).detach().numpy().round(3)

    print_matrix(output_train)
    print()
    print_matrix(output_eval)

solution()

-0.894 0.97 -0.447 0.832
0.894 -0.97 0.447 -0.832

0.136 1.072 0.487 1.135
0.344 0.659 0.592 1.03


In [61]:
a =  np.array([[0.5, 1.0, 0.3, 1.3], [0.9, 0.2, 0.4, 1.0]]) 
t = torch.from_numpy(a).float()
np.repeat(t.var(dim=1), 4).reshape(-1, 4)
t.mean(dim=0)

tensor([0.7000, 0.6000, 0.3500, 1.1500])

In [22]:
a.var(axis=1)

array([0.156875, 0.111875])

In [9]:
np.sqrt(t)

array([[0.70710678, 1.        , 0.54772256, 1.14017543],
       [0.9486833 , 0.4472136 , 0.63245553, 1.        ]])

In [10]:
np.array([0.5, 1.0, 0.3, 1.3]).var()

0.15687500000000001